# P1 - Brian Flores

- All queries must use the ANSI 92 standard for queries with the type safe “on” and will have relational output 

- Output should be in a JSON component. 

Queries should be in one of two formats

1. Medium query should have from 2 to 3 tables joined and use built-in SQL functions and group by summarization. It should include combinations of subqueries or CTE or virtual tables. 

2. Complex query should have from 3 or more tables joined, a custom scalar function, and use built-in SQL functions and group by summarization. It should include combinations of subqueries or CTE or virtual tables. 

<img src="https://static.vecteezy.com/system/resources/previews/022/841/114/non_2x/chatgpt-logo-transparent-background-free-png.png" alt="ChatGPT Logo" width="15" height="15"> **Written in collaboration with ChatGPT from OpenAI to improve understanding and assist with the explanation of the query. And automation of describing queries** <img src="https://static.vecteezy.com/system/resources/previews/022/841/114/non_2x/chatgpt-logo-transparent-background-free-png.png" alt="ChatGPT Logo" width="15" height="15">

___
## Medium 
Q1 

**Proposition:** 

Retrieve a report on product sales, including the total quantity sold, the average selling price, and the top-selling products in each product category.


**Columns:**
1. `ProductCategory`: Aliased column from the `Name` column in the `ProductCategory` table.
2. `ProductName`: Aliased column from the `Name` column in the `Product` table.
3. `NumberOfOrders`: Aggregate function `SUM` applied to the `OrderQty` column from the `SalesOrderDetail` table.
4. `AverageSellingPrice`: Aggregate function `AVG` applied to the `UnitPrice` column from the `SalesOrderDetail` table.
5. `RankInCategory`: Window function `ROW_NUMBER()` applied to generate a rank within each product category based on the sum of order quantities.

**Tables:**
1. `SalesOrderDetail`: It's referenced with the alias `SOD`.
2. `Product`: It's referenced with the alias `P`.
3. `ProductCategory`: It's referenced with the alias `PC`.

**Predicates:**
1. `SOD.ProductID = P.ProductID`: Join predicate that links the `SalesOrderDetail` table with the `Product` table based on the `ProductID` column.
2. `P.ProductSubcategoryID = PC.ProductCategoryID`: Join predicate that links the `Product` table with the `ProductCategory` table based on the `ProductSubcategoryID` and `ProductCategoryID` columns, respectively.

**High lights:**
- Inner join on three seperate tables that only include relavent rows. This is a intersection.
- Group by product category and then specific product helps us get ready for specific Computations within those sections
- Sum() and AVG() help us summarize the groups that have been divided
- Row_Number() is a special window function that uses a set (in this case groups of rows) to compute a function in this case it assigns a number `OVER` a certain set. We want to use rows that are divided by `PC.Name` (This means we restart the enumeration after each different category). We want to then assign a number within the set according to the SUM in the order quantity. Lastly we want to show the top paid products first to show a stakeholder what products sold well according to this rank.
- The logical phases help us understand the query `FROM`, `WHERE`, `GROUP BY`, `SELECT`, `ORDER BY`.


In [ ]:
-- Q1

USE AdventureWorks2017;

SELECT 
    PC.Name AS ProductCategory, 
    P.Name AS ProductName, 
    SUM(SOD.OrderQty) AS NumberOfOrders,
    AVG(SOD.UnitPrice) AverageSellingPrice,
    ROW_NUMBER() OVER (PARTITION BY PC.Name ORDER BY SUM(SOD.OrderQty) DESC) AS RankInCategory

FROM 
    [Sales].[SalesOrderDetail] AS SOD
    INNER JOIN [Production].[Product] AS P 
        ON SOD.ProductID = P.ProductID
    INNER JOIN [Production].[ProductCategory] AS PC
        ON P.ProductSubcategoryID = PC.ProductCategoryID

GROUP BY 
    PC.Name, P.Name
ORDER BY 
    ProductCategory, RankInCategory;





___
## Medium 
Q2 

**Propostion:**
Retrieve a report on sales orders, including the total sales amount, the number of orders per customer, and the average order value. Additionally, identify the top customers based on total sales amount (Do this by Displaying the top sales on top).


**Columns:**
1. `Customerid`: Customer ID from the `Customer` table.
2. `CustomerCompanyName`: Company name of the customer from the `Customer` table.
3. `CustomerContactName`: Contact name of the customer from the `Customer` table.
4. `numberOfOrders`: Number of orders made by each customer, counted using `COUNT`.
5. `totalSalesAmount`: Total sales amount for each customer, calculated as the sum of unit price multiplied by quantity for each order, using `SUM`.
6. `averageSalesAmount`: Average sales amount per order for each customer, calculated as the average of unit price multiplied by quantity for each order, using `AVG`.

**Tables:**
1. `Order`: It's referenced with the alias `O`.
2. `OrderDetail`: It's referenced with the alias `OD`.
3. `Customer`: It's referenced with the alias `C`.

**Predicates:**
1. `O.OrderId = OD.OrderId`: Join predicate that links the `Order` table with the `OrderDetail` table based on the `OrderId` column.
2. `O.CustomerId = C.CustomerId`: Join predicate that links the `Order` table with the `Customer` table based on the `CustomerId` column.

**Highlights:**
- Inner joins
- Group by multiple fields showing that it can be used to clearly define a certain criteria as customer ids can be under different company names and contact names
- Aggregrate function count, sum and AVG which display the level of detail to get an actual sum by multiplying price times quantity.
- Order by which shows top (heavy numbers) first via descending
- The logical phases show that the order by cluase is able to use the column alias to access as it is after the select phase.




In [ ]:
-- Q2 

USE Northwinds2022TSQLV7;

SELECT 
      C.Customerid,
      C.CustomerCompanyName,
      C.CustomerContactName,
      COUNT(O.OrderId) AS numberOfOrders,
      SUM(OD.UnitPrice * OD.Quantity) AS totalSalesAmount,
      AVG(OD.UnitPrice * OD.Quantity) AS averageSalesAmount
FROM
    [Sales].[Order] AS O
INNER JOIN 
    [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId
INNER JOIN 
    [Sales].[Customer] AS C ON O.CustomerId = C.CustomerId

GROUP BY C.CustomerId, C.CustomerCompanyName, C.CustomerContactName

ORDER BY totalSalesAmount DESC;

___
## Medium 
Q3 

**Poroposition:**

Generate a report on suppliers we want to know how many orders the specific supliers have done. It is more than enought to summarize the supplier location and the name as welll as the money generated

**Columns:**
1. `SupplierCity`: City of the supplier from the `Supplier` table.
2. `TotalMoneyGenerated`: Total money generated for each supplier city, calculated as the sum of quantity multiplied by unit price for each product, using `SUM`.
3. `NumberOfOrders`: Number of orders for each supplier city, counted based on the number of distinct `ProductId` values, using `COUNT`.

**Tables:**
1. `Supplier`: It's referenced with the alias `S`.
2. `Product`: It's referenced with the alias `P`.
3. `OrderDetail`: It's referenced with the alias `OD`.

**Predicates:**
1. `P.SupplierId = S.SupplierId`: Join predicate that links the `Product` table with the `Supplier` table based on the `SupplierId` column.
2. `OD.ProductId = P.ProductId`: Join predicate that links the `OrderDetail` table with the `Product` table based on the `ProductId` column.

**Highlights:**
- Several Inner Joins
- Grouping by the city that the supplier serviced
- This enabled us to count how many orders were placed as well the money generated by using these suppliers
- Lastly the information generated is ordered to display high paying suppliers first.



In [ ]:
USE Northwinds2022TSQLV7;

SELECT  
    S.SupplierCity,
    SUM (OD.Quantity * OD.UnitPrice) AS TotalMoneyGenerated,
    COUNT(P.ProductId) AS NumberOfOrders
FROM 
    [Production].[Supplier] AS S
INNER JOIN 
    [Production].[Product] AS P ON P.SupplierId = S.SupplierId
INNER JOIN 
    [Sales].[OrderDetail] AS OD ON  OD.ProductId = P.ProductId
GROUP BY 
    S.SupplierCity
ORDER BY 
    TotalMoneyGenerated DESC;


* * *

## Medium

Q4

**Proposition:**
Create a CTE that captures the relationship between Sales.Order and Sales.OrderDetail. This CTE should display all values of Order and the total quantity for each orderid as well as the total sum for each product id. Label this CTE as OrderValues. Using this same CTE find ther orderid customerId the value and its percentage against every other product that was bought by the same customer

**Columns:**
1. `OrderId`: Order ID from the `Order` table.
2. `CustomerId`: Customer ID from the `Order` table.
3. `val`: Total value of the order, calculated as the sum of the product of quantity, unit price, and discount percentage subtracted from 1 for each order, using `SUM`.
4. `valInPercent`: Percentage representation of the order value relative to the total order values for each customer.

**Tables:**
1. `Order`: It's referenced with the alias `O`.
2. `OrderDetail`: It's referenced with the alias `OD`.

**Predicates:**
1. `O.OrderId = OD.OrderId`: Join predicate that links the `Order` table with the `OrderDetail` table based on the `OrderId` column.


**Highlights:**
- Common Table expression: To create a custom virtual table that summaraizes the relationship between Order and orderdetail
- Inner JOINs
- Subqueries using the same CTE that uses a aggregate function to summarize a value upon this self reference.
- The reusability of this CTE shows a huge benefit of using them in the first place.

In [ ]:
-- Q4 
USE Northwinds2022TSQLV7;

-- A common table expresion that summeraizes the relationship between Order and order detail.
--      That has values of any order and actual values associated with it.
WITH [OrderValues] AS
(
    SELECT 
        O.OrderId,
        O.CustomerId,
        O.EmployeeId,
        O.ShipperId,
        O.OrderDate,
        O.RequiredDate,
        O.ShipToDate,
        SUM(OD.Quantity) AS qty,
        CAST( SUM( OD.Quantity * OD.UnitPrice * (1 - OD.DiscountPercentage) ) AS numeric(12, 2) ) AS val
    FROM 
        [Sales].[Order] AS O
    INNER JOIN 
        [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId
    GROUP BY
        O.OrderId,
        O.CustomerId,
        O.EmployeeId,
        O.ShipperId,
        O.OrderDate,
        O.RequiredDate,
        O.ShipToDate
) 
SELECT 
    OrderId, 
    CustomerId, 
    val, 
    CAST(100.0 * val / (SELECT SUM(O2.val)
                        FROM OrderValues AS O2
                        WHERE O1.CustomerId = O2.CustomerId) 
        AS numeric(5,2)) AS valInPercent
FROM OrderValues as O1;

* * *

## Medium

Q5

**Proposition:**
Return the previous product a customer ordered by date i.e. a customer their product that date and Any Date purchased on beforehand that same product. SImplify this query via a CTE produce the format
Contact name, OrderDate, ProductName, previousDateBoughtSameProduct

**Columns:**
1. `CustomerContactName`: Contact name of the customer from the `Customer` table.
2. `OrderDate`: Order date from the `Order` table.
3. `ProductName`: Name of the product from the `Product` table.
4. `PreviousDateProductBought`: The date of the previous purchase of the same product by the same customer.

**Tables:**
1. `Customer`: It's referenced with the alias `C`.
2. `Product`: It's referenced with the alias `P`.
3. `OrderDetail`: It's referenced with the alias `OD`.
4. `Order`: It's referenced with the alias `O`.

**CTE (Common Table Expression):**
1. `[ProductInformation]`: It selects customer ID, order date, product name, and product ID from the `Product`, `OrderDetail`, and `Order` tables, joining them appropriately.

**Predicates:**
1. `P.ProductId = OD.ProductId`: Join predicate that links the `Product` table with the `OrderDetail` table based on the `ProductId` column.
2. `OD.OrderId = O.OrderId`: Join predicate that links the `OrderDetail` table with the `Order` table based on the `OrderId` column.
3. `C.CustomerId = PI1.CustomerId`: Join predicate that links the `Customer` table with the `ProductInformation` CTE based on the `CustomerId` column.
4. Conditions in the subquery:
   - `C.CustomerId = PI2.CustomerId`: Ensures the previous purchase was made by the same customer.
   - `PI1.OrderDate > PI2.OrderDate`: Ensures the previous purchase date is earlier than the current purchase date.
   - `PI2.ProductId = PI1.ProductID`: Matches the previous purchase with the same product as the current one.


**Highlights**
-  Common Table Expression (CTE): The query uses a CTE named `ProductInformation` to gather information about customers, order dates, product names, and product IDs. This CTE is then used in the main query.

- Join Operations: Several join operations are used to connect tables in the database. The `Product` table is joined with the `OrderDetail` table on the `ProductId` column, and the result is further joined with the `Order` table on the `OrderId` column.

- Column Selection: The main query selects columns from the `Customer` table (`CustomerContactName`) and from the `ProductInformation` CTE (`OrderDate`, `ProductName`). Additionally, it calculates the `PreviousDateProductBought` column, which represents the maximum order date for each customer where a product with the same ID was bought before the current order date.

-  Subquery: The `PreviousDateProductBought` column is calculated using a subquery that retrieves the maximum order date from the `ProductInformation` CTE, meeting specific conditions related to the customer ID, order date, and product ID.


- The query itself allows us to check when was the last time the customer bought a same product at a different but latest time.

In [ ]:
-- Q5 

USE Northwinds2022TSQLV7;

WITH [ProductInformation] AS
(
    SELECT 
        O.CustomerId,
        O.OrderDate,
        P.ProductName,
        P.ProductId
    FROM 
        [Production].[Product] AS P
    INNER JOIN 
        [Sales].[OrderDetail] AS OD ON P.ProductId = OD.ProductId
    INNER JOIN
        [Sales].[Order] AS O ON OD.OrderId = O.OrderId
)

SELECT 
    C.CustomerContactName, 
    PI1.OrderDate,
    PI1.ProductName,
    (
        SELECT 
            MAX(PI2.OrderDate)
        FROM 
            [ProductInformation] AS PI2
        WHERE 
            C.CustomerId = PI2.CustomerId AND
            PI1.OrderDate > PI2.OrderDate AND
            PI2.ProductId = PI1.ProductID
    ) AS PreviousDateProductBought

FROM 
    [Sales].[Customer] AS C
INNER JOIN 
    [ProductInformation] AS PI1 ON C.CustomerId = PI1.CustomerId;

* * *

## Medium

Q6

Show a running sales on order of all times. Display the order id the order date the profit made from the and finally the total running profit made since that date. Display this according to the upToDate profit which should impliclity denote date times as well.


**Columns:**
1. `OrderId`: Order ID from the `Order` table.
2. `OrderDate`: Order date from the `Order` table.
3. `currentProfit`: Current profit calculated as the sum of the product of unit price and quantity for each order, using `SUM`.
4. `UpToDateProfit`: Total profit up to the current order date, calculated by summing the `currentProfit` for all orders up to the current order date.

**Tables:**
1. `Order`: It's referenced with the alias `O`.
2. `OrderDetail`: It's referenced with the alias `OD`.

**CTE (Common Table Expression):**
1. `[ProductInformation]`: It selects order ID, order date, and current profit calculated as the sum of the product of unit price and quantity for each order.

**Predicates:**
1. `O.OrderId = OD.OrderId`: Join predicate that links the `Order` table with the `OrderDetail` table based on the `OrderId` column.



**Highlights:**

-  Common Table Expression (CTE): The query uses a CTE named `ProductInformation` to aggregate data from the `Order` and `OrderDetail` tables. It calculates the `currentProfit` by summing the product of unit price and quantity for each order, grouped by `OrderId` and `OrderDate`.

- Subquery in SELECT Clause: The main query calculates the `UpToDateProfit` column using a correlated subquery. This subquery calculates the cumulative sum of `currentProfit` up to the current `OrderDate` for each row.

- Correlated Subquery: The subquery references the `ProductInformation` CTE and filters rows where the `OrderDate` is less than or equal to the `OrderDate` of the current row in the outer query. This results in a running total of profits for each order.


- Ordering: The final result set is ordered by the `UpToDateProfit`, which represents the cumulative profit up to the current date for each order.


In [ ]:
-- Q6 
USE Northwinds2022TSQLV7;


WITH [ProductInformation] AS
(
    SELECT 
        O.OrderId,
        O.OrderDate,
        SUM(OD.UnitPrice * OD.Quantity) AS currentProfit
    FROM 
        [SALES].[Order] AS O
    INNER JOIN
        [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId     
    GROUP BY 
        O.OrderId, O.OrderDate
)

SELECT 
    PI1.OrderId,
    PI1.OrderDate,
    PI1.currentProfit,
    (SELECT SUM(currentProfit)
    FROM ProductInformation AS PI2
    WHERE PI2.OrderDate <= PI1.OrderDate) AS UpToDateProfit
FROM ProductInformation AS PI1
ORDER BY UpToDateProfit
;


* * *

## Medium

Q7

**Proposition:**

Return the table that shwos the difference in days that have passed since the customer has placed an order. Display this by customer id the date and any order id.

**Columns:**
1. `CustomerId`: Customer ID from the `Order` table.
2. `OrderDate`: Order date from the `Order` table.
3. `OrderId`: Order ID from the `Order` table.
4. `differenceInDays`: The difference in days between the current order date and the previous order date for the same customer.

**Tables:**
1. `Order`: It's referenced with the alias `O1` in the main query and `O2` in the subquery.

**Predicates:**
1. Subquery conditions:
   - `O2.CustomerId = O1.CustomerId`: Ensures the orders belong to the same customer.
   - `(O2.OrderDate = O1.OrderDate AND O2.OrderId < O1.OrderId) OR O2.OrderDate < O1.OrderDate`: Filters orders that are either from the same day but with an earlier order ID or from a previous day.
   - `ORDER BY O2.OrderDate DESC, O2.OrderId DESC`: Orders the results by order date and order ID in descending order, ensuring the top-most record in the subquery is the most recent previous order.



**HighLights:**

- DateDiff(metric, date1, date2) utilizes a subquery to get a metric between 2 dates. In this case utilizes a subquery to get a valid result.
- Uses complex tiebreaker condition that models real world transactions where a order date can be late
- Displays the ordering to show customerids the orderdates and the ids.  

In [ ]:
-- Q7 

USE Northwinds2022TSQLV7;

SELECT 
    O1.CustomerId,
    O1.OrderDate,
    O1.OrderId,
    DATEDIFF(day, 
       (SELECT TOP (1) O2.OrderDate
        FROM [Sales].[Order] AS O2
        WHERE O2.CustomerId = O1.CustomerId 
            AND ( (O2.OrderDate = O1.OrderDate AND O2.OrderId < O1.OrderId) 
                OR O2.OrderDate < O1.OrderDate)
        ORDER BY O2.OrderDate DESC, O2.OrderId DESC        
        )
    , OrderDate) as differenceInDays
FROM 
    [Sales].[Order] AS O1
ORDER BY CustomerId, OrderDate, orderid
;


* * *

## Medium

Q8

**Propositions:**

We want to find our Customers who are doing a fantastic purchases. In order to find this first find the maximum profit and miniumum profit and divide it by 2. This will be the criteria for outstanding customers who made a totaleSale greater than this. Display the CustomerId the region the sales Amount and the delta representing how well of a difference the Customer achieved according to the criteria (Note this will all be positive Numbers!).


**Columns:**
1. `CustomerId`: Customer ID from the `Order` table.
2. `CustomerRegion`: Region of the customer from the `Customer` table.
3. `TotalSalesAmount`: Total sales amount for each order, calculated as the product of unit price and quantity from the `OrderDetail` table.
4. `Delta`: The difference between the total sales amount and half of the difference between the maximum and minimum profits.

**Tables:**
1. `Order`: It's referenced with the alias `O`.
2. `OrderDetail`: It's referenced with the alias `OD`.
3. `Customer`: It's referenced with the alias `C`.

**Variables:**
1. `@MaxProfit`: Variable holding the maximum profit calculated by selecting the top record from `OrderDetail` table ordered by profit in descending order.
2. `@MinProfit`: Variable holding the minimum profit calculated by selecting the top record from `OrderDetail` table ordered by profit in ascending order.

**CTE (Common Table Expression):**
1. `OutstandingCustomers`: It selects customer ID, customer region, total sales amount, and calculates the delta based on the conditions specified.

**Predicates:**
1. `O.OrderId = OD.OrderId`: Join predicate that links the `Order` table with the `OrderDetail` table based on the `OrderId` column.
2. `O.CustomerId = C.CustomerId`: Join predicate that links the `Order` table with the `Customer` table based on the `CustomerId` column.
3. `((OD.UnitPrice * OD.Quantity) - ((@MaxProfit - @MinProfit) / 2))`: Condition ensuring that the total sales amount is greater than or equal to half of the difference between the maximum and minimum profits.



**Highlights**
- Subqueries that find the maximum and minumum criteria that are stored in variables for readability
- A CTE that joins 3 tables via inner joins. This same CTE also finds out the outliers who make outstanding customers.

In [ ]:
-- Q8 

USE Northwinds2022TSQLV7;

DECLARE @MaxProfit AS INT = (
     SELECT TOP (1) (Quantity * UnitPrice)
     FROM [Sales].[OrderDetail]
     ORDER BY (Quantity * UnitPrice) DESC
);

DECLARE @MinProfit AS INT = (
     SELECT TOP (1) (Quantity * UnitPrice)
     FROM [Sales].[OrderDetail]
     ORDER BY (Quantity * UnitPrice) ASC
);


WITH OutstandingCustomers AS (
    SELECT
        O.CustomerId,
        C.CustomerRegion,
        (OD.UnitPrice * OD.Quantity) AS TotalSalesAmount,
        ((OD.UnitPrice * OD.Quantity) - ( (@MaxProfit - @MinProfit) / 2 )) AS Delta
    FROM
        [Sales].[Order] AS O
    INNER JOIN
        [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId
    INNER JOIN
        [Sales].[Customer] AS C ON O.CustomerId = C.CustomerId
    WHERE
        (OD.UnitPrice * OD.Quantity) >= (@MaxProfit - @MinProfit) / 2 
)

SELECT *
FROM [OutstandingCustomers];


* * *

## Medium

Q9

**Proposition:** 

Retrieve a detailed sales report, including the total sales amount, the number of orders, and the average order value for each product category. Additionally, identify the top-selling products in each category based on total sales amount.


**Columns:**
1. `CategoryName`: Name of the product category from the `Category` table.
2. `NumberOfOrders`: Number of orders for each product category, counted based on the number of distinct `OrderId` values.
3. `TotalSalesAmount`: Total sales amount for each product category, calculated as the sum of the product of quantity and unit price for each order.
4. `AverageSellingPrice`: Average selling price for each product category, calculated as the average of the product of quantity and unit price for each order.

**Tables:**
1. `Order`: It's referenced with the alias `O`.
2. `OrderDetail`: It's referenced with the alias `OD`.
3. `Product`: It's referenced with the alias `P`.
4. `Category`: It's referenced with the alias `C`.

**Predicates:**
1. `O.OrderId = OD.OrderId`: Join predicate that links the `Order` table with the `OrderDetail` table based on the `OrderId` column.
2. `P.ProductId = OD.ProductId`: Join predicate that links the `Product` table with the `OrderDetail` table based on the `ProductId` column.
3. `P.CategoryId = C.CategoryId`: Join predicate that links the `Product` table with the `Category` table based on the `CategoryId` column.




**HighLights:**
- Uses agregrate functions
- Group by categories
- Inner Joins
- Presents according to the categoryName

In [ ]:
-- Q9 

USE Northwinds2022TSQLV7;

SELECT 
    C.CategoryName, 
    COUNT(O.OrderId) AS NumberOfOrders,
    SUM(OD.Quantity * OD.UnitPrice) AS TotalSalesAmount,
    AVG(OD.Quantity * OD.UnitPrice) AS AverageSellingPrice
FROM 
    [Sales].[Order] AS O
INNER JOIN
    [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId
INNER JOIN
    [Production].[Product] AS P ON P.ProductId = OD.ProductId
INNER JOIN 
    [Production].[Category] AS C On P.CategoryId = C.CategoryId

GROUP BY 
    C.CategoryName
ORDER BY
    CategoryName, AverageSellingPrice
;


___
## Medium 
Q10 

**Proposition:**

FInd the total sales generated per each customer and find out how much each category has sold.

**Columns:**
1. `CustomerKey`: Key identifying the customer from the `DimCustomer` table.
2. `CategoryName`: English name of the product category from the `DimProductCategory` table.
3. `CustomerTotalOnCategory`: Total product cost for each customer within a specific product category.
4. `totalRevunuePerCategory`: Total revenue generated per product category.

**Tables:**
1. `DimCustomer`: It's referenced with the alias `C`.
2. `FactInternetSales`: It's referenced with the alias `FIS`.
3. `FactSurveyResponse`: It's referenced with the alias `FSR`.
4. `DimProductCategory`: It's referenced with the alias `DPC`.

**CTEs (Common Table Expressions):**
1. `SalesSummary`: It calculates the total product cost for each customer within each product category.
2. `CategorySummary`: It calculates the total revenue generated per product category based on the total product cost for each customer.

**Predicates:**
1. `FIS.CustomerKey = C.CustomerKey`: Join predicate that links the `FactInternetSales` table with the `DimCustomer` table based on the `CustomerKey` column.
2. `FSR.CustomerKey = C.CustomerKey`: Join predicate that links the `FactSurveyResponse` table with the `DimCustomer` table based on the `CustomerKey` column.
3. `DPC.ProductCategoryKey = FSR.ProductCategoryKey`: Join predicate that links the `DimProductCategory` table with the `FactSurveyResponse` table based on the `ProductCategoryKey` column.


**Highlights**
- Multiple CTEs defined
- Inner joins to trace a specific category to a pruchase the customer made
- Summarization for a customer was provided for each category they participated in and then a summatization was provieded for each unique category. 




In [ ]:
-- Q10 

USE AdventureWorksDW2017;


WITH [SalesSummary] AS
(
    SELECT 
        C.CustomerKey,
        DPC.EnglishProductCategoryName AS CategoryName, 
        SUM(FIS.TotalProductCost) AS CustomerTotalOnCategory
    FROM 
        [dbo].[DimCustomer] AS C
    INNER JOIN 
        [dbo].[FactInternetSales] AS FIS ON FIS.CustomerKey = C.CustomerKey
    INNER JOIN 
        [dbo].[FactSurveyResponse] AS FSR ON FSR.CustomerKey = C.CustomerKey
    INNER JOIN 
        [dbo].[DimProductCategory] AS DPC ON DPC.ProductCategoryKey = FSR.ProductCategoryKey
    GROUP BY 
        C.CustomerKey, DPC.EnglishProductCategoryName
),
[CategorySummary] AS
(
    SELECT 
        CategoryName, 
        CAST(SUM(CustomerTotalOnCategory) AS MONEY) AS totalRevunuePerCategory
    FROM 
        SalesSummary 
    GROUP BY 
        CategoryName
)

SELECT 
    CategoryName,
    totalRevunuePerCategory
FROM CategorySummary
ORDER BY totalRevunuePerCategory DESC;


___
## Medium 
Q11 

**Proposition:**

Find employees who have not been audited Give information about their Hire date and name 

**Columns:**
1. `EmployeeFirstName`: First name of the employee from the `Employee` table.
2. `EmployeeLastName`: Last name of the employee from the `Employee` table.
3. `HireDate`: Hire date of the employee from the `Employee` table.
4. `DEPT`: Department of the employee, with a fallback to 'Unknown' if not found.

**Tables:**
1. `Employee`: It's referenced with the alias `E`.
2. `AuditTriggeredEmployeeHistory`: It's referenced with the alias `A`.
3. `Employee` (System-Versioned): It's referenced with the alias `SE`.

**CTE (Common Table Expression):**
1. `NonAuditEmployees`: It contains the distinct list of `EmployeeId` that are present in the `Employee` table but not in the `AuditTriggeredEmployeeHistory` table.

**Predicates:**
1. `E.EmployeeId = NAE.EmployeeId`: Join predicate that links the `Employee` table with the `NonAuditEmployees` CTE based on the `EmployeeId` column.
2. `SE.EmployeeId = NAE.EmployeeId`: Join predicate that links the `Employee` (System-Versioned) table with the `NonAuditEmployees` CTE based on the `EmployeeId` column.


**Highlights:**
- CTE that uses a inner set operator to find all employees who have yet to be audited with `Except`
- Inner Join On Employee and CTE to find data about the employees
- Coalesce function that compresses a subquery on the employees to find what department they belong too.

In [ ]:
-- Q11 

USE Northwinds2022TSQLV7;

WITH [NonAuditEmployees] AS
(
    (
        SELECT 
            DISTINCT EmployeeId
        FROM 
            [HumanResources].[Employee]
    )

    EXCEPT 

    (
        SELECT
            DISTINCT EmployeeId
        FROM 
            [Triggered].[AuditTriggeredEmployeeHistory] AS A
    )
)
SELECT 
    E.EmployeeFirstName,
    E.EmployeeLastName,
    E.HireDate,
    COALESCE((
        SELECT Department
        FROM [SystemVersioned].[Employee] AS SE 
        WHERE SE.EmployeeId = NAE.EmployeeId
    ), 'Unknown') AS DEPT
FROM 
    NonAuditEmployees AS NAE
INNER JOIN 
    [HumanResources].[Employee] AS E ON E.EmployeeId = NAE.EmployeeId


___
## Medium 
Q12 

**Proposition:**

Find the Maximum paid employees in each department


**Columns:**
1. `EmployeeFullName`: Full name of the employee from the `Employee` table.
2. `Department`: Department of the employee from the `Employee` table.
3. `topSalary`: Maximum salary in each department.

**Tables:**
1. `Employee` (System-Versioned): It's referenced with the alias `E`.

**CTEs (Common Table Expressions):**
1. `MaxSalariesPerDepartment`: It calculates the maximum salary (`topSalary`) for each department.
2. `EmployeeDigest`: It joins the `Employee` table with the `MaxSalariesPerDepartment` CTE to retrieve employees with the maximum salary in each department.

**Predicates:**
1. `MS.Department = E.Department`: Join predicate that links the `MaxSalariesPerDepartment` CTE with the `Employee` table based on the `Department` column.
2. `E.Salary = MS.topSalary`: Additional condition to ensure that the employee's salary matches the maximum salary for their department.


**Highlights:**
- A CTE that searches for the maximum salary in each department
- A CTE that uses the previous CTE and returns the name and Department and the top salary that the employee has
- The inner join finds the exact employee based on departmnent and salary



In [ ]:
-- Q12 

USE Northwinds2022TSQLV7;

WITH MaxSalariesPerDepartment AS (
    SELECT 
        E.Department,
        MAX(E.Salary) AS topSalary
    FROM 
        [SystemVersioned].[Employee] AS E
    GROUP BY
        E.Department
),
EmployeeDigest AS (
    SELECT 
        E.EmployeeFullName,
        MS.Department,
        MS.topSalary
    FROM 
        [SystemVersioned].[Employee] AS E
    INNER JOIN 
        MaxSalariesPerDepartment AS MS ON 
            MS.Department = E.Department AND
            E.Salary = MS.topSalary
)
SELECT *
FROM EmployeeDigest;



___
## Medium 
Q13 

**Proposition:**

Find all Products that have been shipped from a different location from where they are shipped from include the locations the total profit and the item itself that was shipped


**Columns:**
1. `SourceCity`: City where the product is supplied from, obtained from the `Supplier` table.
2. `DestinationCity`: City where the product is shipped to, obtained from the `Order` table.
3. `ProductName`: Name of the product, obtained from the `Product` table.
4. `TotalProfit`: Total profit generated from the sales transaction, calculated as the sum of the product of unit price, quantity, and discount percentage subtracted from 1, obtained from the `OrderDetail` table.

**Tables:**
1. `OrderDetail`: It's referenced with the alias `OD`.
2. `Order`: It's referenced with the alias `O`.
3. `Shipper`: It's referenced with the alias `S`.
4. `Product`: It's referenced with the alias `P`.
5. `Supplier`: It's referenced with the alias `SP`.

**CTE (Common Table Expression):**
1. `ShippedFromDifferentLocation`: It selects data about the products that are shipped from a different location than they are supplied.

**Predicates:**
1. `OD.OrderId = O.OrderId`: Join predicate that links the `OrderDetail` table with the `Order` table based on the `OrderId` column.
2. `O.ShipperId = S.ShipperId`: Join predicate that links the `Order` table with the `Shipper` table based on the `ShipperId` column.
3. `OD.ProductId = P.ProductId`: Join predicate that links the `OrderDetail` table with the `Product` table based on the `ProductId` column.
4. `SP.SupplierId = P.SupplierId`: Join predicate that links the `Supplier` table with the `Product` table based on the `SupplierId` column.
5. `O.ShipToCity <> SP.SupplierCity`: Condition checking if the shipping destination city is different from the supplier city.


**HighLights:**
 - CTE that was able to abstract the idea of different locations
 - Multiple Inner hoins to connect the order with supplier
 - Use of the not equal operator `<>`
 - Grouped BY operator in order to apply the aggregation of sum.

In [ ]:
-- Q13 
USE Northwinds2022TSQLV7;

WITH [ShippedFromDifferentLocation] AS (
    SELECT 
        SP.SupplierCity AS SourceCity,
        O.ShipToCity AS DestinationCity,
        P.ProductName,
        SUM(OD.UnitPrice * OD.Quantity * (1 - OD.DiscountPercentage)) AS TotalProfit
    FROM 
        [Sales].[OrderDetail] OD
    INNER JOIN 
        [Sales].[Order] O ON OD.OrderId = O.OrderId
    INNER JOIN 
        [Sales].[Shipper] S ON O.ShipperId = S.ShipperId
    INNER JOIN 
        [Production].[Product] P ON OD.ProductId = p.ProductId
    INNER JOIN 
        [Production].[Supplier] AS SP ON SP.SupplierId = P.SupplierId
    WHERE 
        O.ShipToCity <> SP.SupplierCity
    GROUP BY 
         SP.SupplierCity, O.ShipToCity, S.ShipperCompanyName, P.ProductName
)

SELECT 
    ProductName,
    SourceCity,
    DestinationCity,
    TotalProfit
FROM 
    [ShippedFromDifferentLocation];


* * *

## Complex

Q14

**Proposition:**

Retreve a sales report that shows the total sales amount and Average Order amount for each category do this by using a CTE. In order to calculate prices implement a scalar function. Once you have this CTE find all Products that have a total sales amount that are bigger than the average


**Columns:**
1. `CategoryName`: Name of the product category from the `Category` table.
2. `NumberOfOrders`: Number of orders for each product category.
3. `TotalSalesAmount`: Total sales amount for each product category.
4. `AverageOrderValue`: Average order value for each product category.
5. `ProductName`: Name of the product.
6. `Delta`: Difference between the total sales amount of a product and the average order value of its category.

**Tables:**
1. `Order`: It's referenced with the alias `O`.
2. `OrderDetail`: It's referenced with the alias `OD`.
3. `Product`: It's referenced with the alias `P`.
4. `Category`: It's referenced with the alias `C`.

**CTEs (Common Table Expressions):**
1. `SalesSummary`: It calculates general information about products via a sales summary, including the category name, number of orders, total sales amount, and average order value for each category.

**Functions:**
1. `dbo.CalculateTotalPrice`: A user-defined function that calculates the total price of a product based on its quantity and unit price.

**Predicates:**
1. Join predicates: Several join conditions are used to link the `Order`, `OrderDetail`, `Product`, and `Category` tables appropriately.
2. Filtering conditions: Conditions are used to filter the data based on the comparison of total sales amount with average order value within each category.

  

**HIGHLIGHTS:**

- A custom scalar function that returns the multiplication of a quantity and price
- A CTE that holds the basic summary of product categories
- A subquery is provided on a join to merge where products meet the threshold that was provided by the Sales Summary CTE.
- Multiple Inner joins are done in the CTE and in the main query.

In [ ]:
-- Define a custom scalar function to calculate the total price for an order detail that has Qauntity and UnitPrice

CREATE OR ALTER FUNCTION dbo.CalculateTotalPrice (@Quantity INT, @UnitPrice MONEY)
RETURNS MONEY
AS 
    BEGIN
    DECLARE @TotalPrice MONEY;
    SET @TotalPrice = @Quantity * @UnitPrice;
    RETURN @TotalPrice;
    END;
GO

In [ ]:
-- Q14 


USE Northwinds2022TSQLV7;


-- Collect general Info about the products via a Sales Summary
WITH SalesSummary AS (
    SELECT 
        C.CategoryName,
        COUNT(DISTINCT O.OrderId) AS NumberOfOrders,
        SUM(dbo.CalculateTotalPrice(OD.Quantity, OD.UnitPrice)) AS TotalSalesAmount,
        AVG(dbo.CalculateTotalPrice(OD.Quantity, OD.UnitPrice)) AS AverageOrderValue
    FROM 
        Sales.[Order] AS O
    INNER JOIN
        Sales.OrderDetail AS OD ON O.OrderId = OD.OrderId
    INNER JOIN
        Production.Product AS P ON P.ProductId = OD.ProductId
    INNER JOIN 
        Production.Category AS C ON P.CategoryId = C.CategoryId
    GROUP BY 
        C.CategoryName
)
-- combine the Sales report with other products and find those who are above average
SELECT 
    [SS].CategoryName,
    [SS].NumberOfOrders,
    [SS].TotalSalesAmount,
    [SS].AverageOrderValue,
    [CategoryProduct].ProductName,
    ([CategoryProduct].[totalSalesAmount] - [SS].[AverageOrderValue]) as Delta
FROM 
    [SalesSummary] AS SS
LEFT JOIN (
    SELECT 
        C.CategoryName,
        dbo.CalculateTotalPrice(OD.Quantity, OD.UnitPrice) AS totalSalesAmount,
        P.ProductName 
    FROM 
        [Sales].[Order] AS O
    INNER JOIN 
        [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId
    INNER JOIN 
        [Production].[Product] AS P ON P.ProductId = OD.ProductId
    INNER JOIN 
        [Production].[Category] AS C ON C.CategoryId = P.CategoryId
) AS [CategoryProduct] ON [CategoryProduct].totalSalesAmount >= [SS].AverageOrderValue 
                        AND [CategoryProduct].CategoryName = [SS].CategoryName
ORDER BY [SS].[CategoryName], [Delta] DESC
;





___
## Complex 
Q15 


**Proposition:**

Retrieve a detailed report on customer transactions, including the total amount spent by each customer using different payment methods.


**Columns:**
1. `Customer`: Customer name from the `Customer` dimension.
2. `Payment Method`: Payment method name from the `Payment Method` dimension.
3. `TotalAmountSpent`: Total amount spent by each customer using each payment method.

**Tables:**
1. `Customer`: It's referenced with the alias `C` from the `Dimension` schema.
2. `Transaction`: It's referenced with the alias `T` from the `Fact` schema.
3. `Payment Method`: It's referenced with the alias `PM` from the `Dimension` schema.

**CTE (Common Table Expression):**
1. `TransactionTypeSummay`: It aggregates data to summarize the total amount spent by each customer using each payment method.

**Predicates:**
1. `C.[Customer Key] != 0 AND C.[Customer Key] = T.[Customer Key]`: Join predicate that links the `Customer` dimension with the `Transaction` fact table based on the `Customer Key` column.
2. `PM.[Payment Method Key] != 0 AND PM.[Payment Method Key] = T.[Payment Method Key]`: Join predicate that links the `Payment Method` dimension with the `Transaction` fact table based on the `Payment Method Key` column.



**High Lights:**

- Inner joins on 3 tables that check for the databases default unknown values.
- It calculates the total amount spent by each customer (The two and only biggest companies in the database) for each payment method.
- Utilizes a Common Table Expression (CTE) named `CustomerTransactionSummary` to simplify the calculation of total transaction amounts.
- Within the CTE, joins the tables to retrieve relevant information and uses the `SUM()` aggregate function to calculate the total amount spent.
- Presents the results, including customer name, payment method, and total amount spent, ordered by customer and payment method.



In [ ]:
-- Q15 

USE WideWorldImportersDW;


WITH [TransactionTypeSummay] AS
(
    SELECT 
        C.Customer,
        PM.[Payment Method],
        SUM(T.[Total Including Tax]) AS TotalAmountSpent
    FROM 
        [Dimension].[Customer]AS C
    INNER JOIN 
        [Fact].[Transaction] AS T ON 
            C.[Customer Key] != 0 AND
            C.[Customer Key] = T.[Customer Key]
    INNER JOIN
        [Dimension].[Payment Method] AS PM ON 
            PM.[Payment Method Key] != 0 AND
            PM.[Payment Method Key] = T.[Payment Method Key]
    GROUP BY
        C.Customer, 
        PM.[Payment Method]
)
SELECT 
    Customer,
    [Payment Method],
    TotalAmountSpent
FROM 
    TransactionTypeSummay;



___
## Complex 

Q16 

**Proposition:**

Within the wideworldImporters databasecreate a table value function that returns information on a customers purchase the deliveryMethod the item they bought and lastly the total amount spent. Make sure to clean it up by wrapping this complex table in a CTE labeled CustomerTransactionSummary. The function should return the top n purchases that the each customer made via the delivery method.

**Tables:**
1. `[Sales].[Customers]`: Referenced as `C`.
2. `[Sales].[Invoices]`: Referenced as `I`.
3. `[Application].[DeliveryMethods]`: Referenced as `DM`.
4. `[Sales].[OrderLines]`: Referenced as `OL`.
5. `[Warehouse].[StockItems]`: Referenced as `SI`.

**Columns:**
1. `CustomerID`: From the `[Sales].[Customers]` table.
2. `OrderID`: From the `[Sales].[Invoices]` table.
3. `DeliveryMethodID`: From the `[Application].[DeliveryMethods]` table.
4. `StockItemID`: From the `[Warehouse].[StockItems]` table.
5. `TotalAmountSpent`: Calculated using the `SUM(OL.Quantity * OL.UnitPrice)` aggregate function.
6. `PurchaseRank`: Calculated using `ROW_NUMBER() OVER (PARTITION BY C.CustomerID ORDER BY SUM(OL.Quantity * OL.UnitPrice) DESC)`.

**CTEs (Common Table Expressions):**
1. `[CustomerTransactionSummary]`: It summarizes customer transactions by aggregating data from different tables and calculating the total amount spent for each transaction and the purchase rank.

**Predicates:**
1. `C.CustomerID = I.CustomerID`: Joining `[Sales].[Customers]` with `[Sales].[Invoices]` on the `CustomerID` column.
2. `DM.DeliveryMethodID = I.DeliveryMethodID`: Joining `[Application].[DeliveryMethods]` with `[Sales].[Invoices]` on the `DeliveryMethodID` column.
3. `OL.OrderID = I.OrderID`: Joining `[Sales].[OrderLines]` with `[Sales].[Invoices]` on the `OrderID` column.
4. `SI.StockItemID = OL.StockItemID`: Joining `[Warehouse].[StockItems]` with `[Sales].[OrderLines]` on the `StockItemID` column.
5. `PARTITION BY C.CustomerID`: Partitioning the result set by `CustomerID` in the `ROW_NUMBER()` function.



**Highlights:**

- A table value function that takes in a argument called topNOrders
- Inside the return statement it uses a CTE that joins many tables
- Multiple inner joins were done in order to connect a customer to the specific Order and the delivery type
- Applies a window function to determine the rank of a specific purchase.
- Filters all groups outside the CTE for readability.
- The actual call uses correlated subqueries to get names for fields based on the key.




In [ ]:
-- Q16 pt 1 creates a function

USE WideWorldImporters;
GO

CREATE OR ALTER FUNCTION [dbo].[GetTopPurchasesForCustomers] (@topNOrders AS INT)
RETURNS TABLE
AS
RETURN 
(
    WITH [CustomerTransactionSummary] AS
    (   
        SELECT  
            C.CustomerID,
            I.OrderID,
            DM.DeliveryMethodID,
            SI.StockItemID,
            SUM(OL.Quantity * OL.UnitPrice) AS TotalAmountSpent, 
            ROW_NUMBER() OVER (PARTITION BY C.CustomerID ORDER BY SUM(OL.Quantity * OL.UnitPrice) DESC) AS PurchaseRank
        FROM 
            [Sales].[Customers] AS C
        INNER JOIN
            [Sales].[Invoices] AS I ON C.CustomerID = I.CustomerID
        INNER JOIN 
            [Application].[DeliveryMethods] AS DM ON DM.DeliveryMethodID = I.DeliveryMethodID
        INNER JOIN 
            [Sales].[OrderLines] AS OL ON OL.OrderID = I.OrderID 
        INNER JOIN 
            [Warehouse].[StockItems] AS SI ON SI.StockItemID = OL.StockItemID
        GROUP BY
            C.CustomerID,
            I.OrderID,
            DM.DeliveryMethodID, 
            SI.StockItemID
    )
    SELECT 
        CustomerID,
        OrderID,
        DeliveryMethodID,
        StockItemID,
        TotalAmountSpent,
        PurchaseRank
    FROM 
        [CustomerTransactionSummary]
    WHERE 
        PurchaseRank <= @topNOrders
)
GO


**Q16 continued (Application of function)**


**Tables:**
1. `[Sales].[Customers]`: Referenced as `C`.
2. `[Application].[DeliveryMethods]`: Referenced as `DM`.
3. `[Warehouse].[StockItems]`: Referenced as `SI`.

**Columns:**
1. `CustomerName`: From the `[Sales].[Customers]` table.
2. `DeliveryMethodName`: From the `[Application].[DeliveryMethods]` table.
3. `StockItemName`: From the `[Warehouse].[StockItems]` table.
4. `TotalAmountSpent`: From the result set returned by the function `GetTopPurchasesForCustomers`.
5. `PurchaseRank`: From the result set returned by the function `GetTopPurchasesForCustomers`.

**Function:**
1. `GetTopPurchasesForCustomers`: A user-defined function that takes an input parameter `@topNOrders` and returns the top purchases for each customer.

**Predicates:**
1. `C.CustomerID = TPC.CustomerID`: Joining `[Sales].[Customers]` with the result set returned by the function on the `CustomerID` column.
2. `DM.DeliveryMethodID = TPC.DeliveryMethodID`: Joining `[Application].[DeliveryMethods]` with the result set returned by the function on the `DeliveryMethodID` column.
3. `SI.StockItemID = TPC.StockItemID`: Joining `[Warehouse].[StockItems]` with the result set returned by the function on the `StockItemID` column.


In [ ]:
-- Q16 pt 2 executes the function

USE WideWorldImporters;

DECLARE @topNOrders AS INT = 3;

SELECT 
    (
     SELECT
        C.CustomerName
     FROM 
        [Sales].[Customers] AS C
     WHERE 
        C.CustomerID = TPC.CustomerID
    ) AS CustName,
    (
     SELECT 
        DM.DeliveryMethodName
     FROM 
        [Application].[DeliveryMethods] AS DM
     WHERE
        DM.DeliveryMethodID = TPC.DeliveryMethodID
    ) AS DeliveryMethod,
    (
     SELECT 
        SI.StockItemName
     FROM 
        [Warehouse].[StockItems] AS SI
     WHERE
        SI.StockItemID = TPC.StockItemID
    ) AS ItemName,
    TPC. totalAmountSpent,
    TPC.PurchaseRank


FROM [dbo].[GetTopPurchasesForCustomers](@topNOrders) AS TPC;

* * *

## Complex

Q17

**Proposition:**
Now that we know our top customers using `[dbo].[GetTopPurchasesForCustomers](@topNOrders)` we can now get more data about them especially if we want to contact them for more details. Display the top 3 orders per customer but this time introduce name their number thier email address and the indidivual price of the the item


**Tables:**
1. `[Sales].[Customers]`: Referenced as `C`.
2. `[Application].[People]`: Referenced as `P`.
3. `[Application].[DeliveryMethods]`: Referenced as `DM`.
4. `[Warehouse].[StockItems]`: Referenced as `SI`.

**Columns:**
1. `CustomerName`: From the `[Sales].[Customers]` table.
2. `PhoneNumber`: From the `[Sales].[Customers]` table.
3. `EmailAddress`: From the `[Application].[People]` table.
4. `DeliveryMethodName`: From the `[Application].[DeliveryMethods]` table.
5. `StockItemName`: From the `[Warehouse].[StockItems]` table.
6. `UnitPrice`: From the `[Warehouse].[StockItems]` table.
7. `TotalAmountSpent`: From the result set returned by the function `GetTopPurchasesForCustomers`.
8. `PurchaseRank`: From the result set returned by the function `GetTopPurchasesForCustomers`.

**Function:**
1. `GetTopPurchasesForCustomers`: A user-defined function that takes an input parameter `@topNOrders` and returns the top purchases for each customer.

**Predicates:**
1. `C.CustomerID = TPC.CustomerID`: Joining `[Sales].[Customers]` with the result set returned by the function on the `CustomerID` column.
2. `P.PersonID = C.PrimaryContactPersonID`: Joining `[Application].[People]` with `[Sales].[Customers]` on the `PersonID` column to retrieve the email address of the primary contact person.
3. `DM.DeliveryMethodID = TPC.DeliveryMethodID`: Joining `[Application].[DeliveryMethods]` with the result set returned by the function on the `DeliveryMethodID` column.
4. `SI.StockItemID = TPC.StockItemID`: Joining `[Warehouse].[StockItems]` with the result set returned by the function on the `StockItemID` column.


**Highlights:**

- Inner joins to connect values to keys
- A subquery is done to be able to get the persons email
- Underneath all thesequeries is a complex function that has many features such as a TVF and a CTE.

In [ ]:
-- Q17 
USE WideWorldImporters;

DECLARE @topNOrders AS INT = 3;


SELECT 
    C.CustomerName AS CustName,
    C.PhoneNumber AS CustomerPhoneNumber,
    (
        SELECT P.EmailAddress
        FROM [Application].[People] AS P
        WHERE C.PrimaryContactPersonID = P.PersonID
    ) AS CustomerEmailAddress,
    DM.DeliveryMethodName AS DeliveryMethod,
    SI.StockItemName AS ItemName,
    SI.UnitPrice AS ItemUnitPrice,
    TPC.TotalAmountSpent,
    TPC.PurchaseRank
FROM 
    [dbo].[GetTopPurchasesForCustomers](@topNOrders) AS TPC
INNER JOIN 
    [Sales].[Customers] AS C ON C.CustomerID = TPC.CustomerID
INNER JOIN 
    [Application].[DeliveryMethods] AS DM ON DM.DeliveryMethodID = TPC.DeliveryMethodID
INNER JOIN 
    [Warehouse].[StockItems] AS SI ON SI.StockItemID = TPC.StockItemID;


___
## Complex 
Q18 

**Proposition:**

Create A custom function that takes in a  desired category from the following. 
- Other Wholesaler
- Novelty Goods Supplier
- Toy Supplier
- Clothing Supplier
- Packaging Supplier
- Courier Services Supplier
- Financial Services Supplier
- Marketing Services Supplier
- Insurance Services Supplier

Additionally provide a number that picks the top N customers who made purchases in the category.

For example this function finds purchases under the Novelty Goods Supplier category and selects the top 50 customers who made purchases in this category.


**Columns:**
1. `CustomerID`: From the `[Sales].[Customers]` table.
2. `OrderID`: From the `[Sales].[Orders]` table.
3. `SupplierCategoryName`: From the `[Purchasing].[SupplierCategories]` table.
4. `TotalAmountSpent`: Calculated using `SUM(OL.Quantity * OL.UnitPrice)`.


**Tables:**
1. `[Sales].[Customers]`: Referenced as `C`.
2. `[Sales].[Orders]`: Referenced as `O`.
3. `[Sales].[OrderLines]`: Referenced as `OL`.
4. `[Purchasing].[PurchaseOrderLines]`: Referenced as `POL`.
5. `[Warehouse].[StockItems]`: Referenced as `SI`.
6. `[Purchasing].[Suppliers]`: Referenced as `S`.
7. `[Purchasing].[SupplierCategories]`: Referenced as `SC`.

**CTE (Common Table Expression):**
1. `[TopOrdersUnderProductCategory]`: It selects the top N orders under a specific product category. The CTE returns the customer ID, order ID, supplier category name, and total amount spent for each order.

**Predicates:**
1. `C.CustomerID = O.CustomerID`: Joining `[Sales].[Customers]` with `[Sales].[Orders]` on the `CustomerID` column.
2. `OL.OrderID = O.OrderID`: Joining `[Sales].[OrderLines]` with `[Sales].[Orders]` on the `OrderID` column.
3. `POL.PurchaseOrderID = O.OrderID`: Joining `[Purchasing].[PurchaseOrderLines]` with `[Sales].[Orders]` on the `PurchaseOrderID` column.
4. `SI.StockItemID = POL.StockItemID`: Joining `[Warehouse].[StockItems]` with `[Purchasing].[PurchaseOrderLines]` on the `StockItemID` column.
5. `S.SupplierID = SI.SupplierID`: Joining `[Purchasing].[Suppliers]` with `[Warehouse].[StockItems]` on the `SupplierID` column.
6. `SC.SupplierCategoryID = S.SupplierCategoryID`: Joining `[Purchasing].[SupplierCategories]` with `[Purchasing].[Suppliers]` on the `SupplierCategoryID` column.
7. `SC.SupplierCategoryName = @desiredCategory`: Filtering for a specific supplier category name.



**Highlights:**
- A Table valued function thate takes in 2 parameters
- A CTE is applied for readability and maintance
- Multiple Inner Joins are done to obtain the product category that a customer made
- A group by cluase is provided to be able to crunch numbers on the profit made under this category.
- The combination of Order by and TOP enables us to get the "Top customers" in this category which is powerful enough to do other queries if we want to contact these customers.


In [ ]:
-- Q18 

USE WideWorldImporters;
GO

CREATE OR ALTER FUNCTION [dbo].[topNPurchasesOnCategory] 
    (@desiredCategory NVARCHAR(100), @topNOrders INT)
RETURNS TABLE
AS
RETURN 
(
    WITH [TopOrdersUnderProductCategory] AS 
    (
        SELECT 
        TOP (@topNOrders) 
            C.CustomerId,
            O.OrderID,
            SC.SupplierCategoryName,
            SUM(OL.Quantity * OL.UnitPrice) AS TotalAmountSpent
        FROM 
            [Sales].[Customers] AS C
        INNER JOIN 
            [Sales].[Orders] AS O ON C.CustomerID = O.CustomerID
        INNER JOIN 
            [Sales].[OrderLines] AS OL ON OL.OrderID = O.OrderID
        INNER JOIN 
            [Purchasing].[PurchaseOrderLines] AS POL ON POL.PurchaseOrderID = O.OrderID
        INNER JOIN 
            [Warehouse].[StockItems] AS SI ON SI.StockItemID = POL.StockItemID
        INNER JOIN 
            [Purchasing].[Suppliers] AS S ON S.SupplierID = SI.SupplierID
        INNER JOIN 
            [Purchasing].[SupplierCategories] AS SC ON SC.SupplierCategoryID = S.SupplierCategoryID
        WHERE
            SC.SupplierCategoryName = @desiredCategory
        GROUP BY 
            C.CustomerID, O.OrderID, S.SupplierID, SC.SupplierCategoryName
        ORDER BY
            TotalAmountSpent DESC
    )
    SELECT
        CustomerID,
        OrderID,
        SupplierCategoryName,
        TotalAmountSpent
    FROM 
        TopOrdersUnderProductCategory
)
GO

___
## Complex 
Q19 

**Proposition:**

Find  meta data about a customer (Name, email, country)within a certain category. Generalize this by using variables. 


**Columns:**
1. `CustomerName`: From the `[Sales].[Customers]` table.
2. `EmailAddress`: From the `[Application].[People]` table.
3. `SupplierCategoryName`: From the result set returned by the function `topNPurchasesOnCategory`.
4. `TotalAmountSpent`: From the result set returned by the function `topNPurchasesOnCategory`.

**Tables:**
1. `[Sales].[Customers]`: Referenced as `C`.
2. `[Application].[People]`: Referenced as `P`.


**Predicates:**
1. `C.CustomerID = TPC.CustomerID`: Joining `[Sales].[Customers]` with the result set returned by the function on the `CustomerID` column.
2. `P.PersonID = C.PrimaryContactPersonID`: Joining `[Application].[People]` with `[Sales].[Customers]` on the `PersonID` column to retrieve the email address of the primary contact person.
3. `TPC.SupplierCategoryName = @desiredCategory`: Filtering the result set returned by the function for a specific supplier category name.
4. `C.CustomerName NOT LIKE '%(%'`: Filtering out customer names containing parentheses.


**Highlights:**
- A Table valued function thate takes in 2 parameters
- Multiple Inner joins to find the customers name and email address
- A regular expresion designed to filter out company names and find actual people.


In [ ]:
-- Q19 
USE WideWorldImporters;

DECLARE @desiredCategory AS NVARCHAR(100) = N'Clothing Supplier';
DECLARE @topNOrders AS INT = 500;

SELECT 
    C.CustomerName,
    P.EmailAddress,
    TPC.SupplierCategoryName,
    TPC.TotalAmountSpent
FROM 
    [dbo].[topNPurchasesOnCategory](@desiredCategory, @topNOrders) AS TPC
INNER JOIN 
    [Sales].[Customers] AS C ON C.CustomerID = TPC.CustomerID
INNER JOIN 
    [Application].[People] AS P ON P.PersonID = C.PrimaryContactPersonID
WHERE 
 C.CustomerName NOT LIKE '%(%'

* * *

## Complex

Q20

**Proposition:**

We want to send a special discount to customers who have been in the top $$M_{I}$$  of each $$N_{I}$$ category. However we would also like to know a specific trhreshold for who to send a discount to. A number greater or equal to 2 would be a great criteria to find. For example we would like to send discounts who purchased from a (Novelty Goods Supplier) they were in the top 500, the same customer should be have purchased from a (Toy Supplier) and was in the top 600. This same customer has bought from a (Clothing Supplier) and was in the top 700. And then Find the intersection of this result set and then determine based on a count who should be given the discount.

It i possible to have customers who only made purchases from one supplier if the dataset is small.



**Columns:**
1. `CustomerName`: From the `[Sales].[Customers]` table.
2. `EmailAddress`: From the `[Application].[People]` table.
3. `TotalAmountSpent`: Calculated using the function `SUM(CWM.TotalAmountSpent)` for each loyal customer.

**Tables:**
1. `[Sales].[Customers]`: Referenced as `C`.
2. `[Application].[People]`: Referenced as `P`.

**CTEs (Common Table Expressions):**
1. `[TopCustomerCriteria]`: It defines the criteria for identifying top customers in various categories along with the threshold for being considered loyal.
2. `[AllCustomersWhoMatchedCriteria]`: It selects all customers who meet the criteria defined in `[TopCustomerCriteria]` by calling the user-defined function `[dbo].[topNPurchasesOnCategory]`.
3. `[LoyalCustomers]`: It groups the customers identified in `[AllCustomersWhoMatchedCriteria]` and filters those who meet the loyalty threshold specified by `@LoyalCustomerThreshold`.


**Predicates:**
1. `C.CustomerID = LC.CustomerID`: Joining `[Sales].[Customers]` with the `LoyalCustomers` CTE on the `CustomerID` column.
2. `P.PersonID = C.PrimaryContactPersonID`: Joining `[Application].[People]` with `[Sales].[Customers]` on the `PersonID` column to retrieve the email address of the primary contact person.
3. `C.CustomerName NOT LIKE '%(%'`: Filtering out customer names containing parentheses.




**HighLights:**

- A virtual table idenitified as a CTE that cleverly uses Union all to capture the essence of a virtual table.
- Then once this table is generated it applies a custom function using the table as input! This effectively acts as a dynamic union operator without explicilty writing out.
- Once all of the of the tables have been added into the result set we will find out who should be emailed the discount
- We cant dynamically take the intersection so we will count the membership of the customer respective with the number of supplier fields they have participated in. This in effects checks the cardinality of the membership of a customer which we will check if it is greater or equal to the discount criteria.
- Once we find those customers we will find customers who are not companies (we want to target normal everyday customers). via the final where clause
- We also want to find meta data of the customer such as their name, their email and the total amount they spent across the categories.
- Multiple CTEs were created which speak for themselves.




In [ ]:
-- Q20 
USE WideWorldImporters;

DECLARE @LoyalCustomerThreshold AS INT = 2;

-- Define the criteria we are after
WITH [TopCustomerCriteria] AS (
    SELECT 'Novelty Goods Supplier' AS Category, 700 AS TopN
    UNION ALL
    SELECT 'Toy Supplier', 800
    UNION ALL
    SELECT 'Clothing Supplier', 900
    UNION ALL
    SELECT 'Packaging Supplier', 500
),
-- Add those customers using the custom function that enables us to find top customers
[AllCustomersWHoMatchedCriteria] AS
(
    SELECT  
        *
    FROM 
        TopCustomerCriteria  AS TCC
    CROSS APPLY 
        [dbo].[topNPurchasesOnCategory](TCC.Category, TCC.TopN)
),
-- Grouping and then counting shows the cardinality of membership which can be applied with the loyalCustomerThreshold
[LoyalCustomers] AS (
    SELECT 
        CustomerID
    FROM 
        AllCustomersWHoMatchedCriteria AS CWM
    GROUP BY 
        CustomerID
    HAVING 
        COUNT(DISTINCT Category) >= @LoyalCustomerThreshold
)

SELECT 
    C.CustomerName,
    P.EmailAddress,
    (
        SELECT SUM(CWM.TotalAmountSpent)
        FROM AllCustomersWHoMatchedCriteria AS CWM
        INNER JOIN LoyalCustomers AS C ON C.CustomerID = CWM.CustomerID
        GROUP BY C.CustomerID
        HAVING C.CustomerID = LC.CustomerID
    ) AS totalAmountSpent
FROM 
    LoyalCustomers AS LC
INNER JOIN 
    [Sales].[Customers] AS C ON C.CustomerID = LC.CustomerID
INNER JOIN 
    [Application].[People] AS P ON P.PersonID = C.PrimaryContactPersonID
WHERE 
    C.CustomerName NOT LIKE '%(%'
ORDER BY 
    totalAmountSpent DESC
;

---


# Presentation Queries TOP BEST!

**TOP BEST**

**<u>TQ 1</u>**

**Proposition**:

Retrieve a report on product sales, including the total quantity sold, the average selling price, and the top-selling products in each product category.

**High lights:**

- Inner join on three seperate tables that only include relavent rows. This is a intersection.
- Group by product category and then specific product helps us get ready for specific Computations within those sections
Sum() and AVG() help us summarize the groups that have been divided
- Row_Number() is a special window function that uses a set (in this case groups of rows) to compute a function in this case it assigns a number OVER a certain set. We want to use rows that are divided by PC.Name (This means we restart the enumeration after each different category). We want to then assign a number within the set according to the SUM in the order quantity. Lastly we want to show the top paid products first to show a stakeholder what products sold well according to this rank.
- The logical phases help us understand the query FROM, WHERE, GROUP BY, SELECT, ORDER BY.

In [ ]:
-- TQ1 , Q1

USE AdventureWorks2017;

SELECT 
    PC.Name AS ProductCategory, 
    P.Name AS ProductName, 
    SUM(SOD.OrderQty) AS NumberOfOrders,
    AVG(SOD.UnitPrice) AverageSellingPrice,
    ROW_NUMBER() OVER (PARTITION BY PC.Name ORDER BY SUM(SOD.OrderQty) DESC) AS RankInCategory

FROM 
    [Sales].[SalesOrderDetail] AS SOD
    INNER JOIN [Production].[Product] AS P 
        ON SOD.ProductID = P.ProductID
    INNER JOIN [Production].[ProductCategory] AS PC
        ON P.ProductSubcategoryID = PC.ProductCategoryID

GROUP BY 
    PC.Name, P.Name
ORDER BY 
    ProductCategory, RankInCategory
-- FOR JSON AUTO, INCLUDE_NULL_VALUES ;


***
**<u>TQ 2</u>**


**Proposition:**

Create A custom function that takes in a  desired category from the following. 
- Other Wholesaler
- Novelty Goods Supplier
- Toy Supplier
- Clothing Supplier
- Packaging Supplier
- Courier Services Supplier
- Financial Services Supplier
- Marketing Services Supplier
- Insurance Services Supplier

Additionally provide a number that picks the top N customers who made purchases in the category.

For example this function finds purchases under the Novelty Goods Supplier category and selects the top 50 customers who made purchases in this category.

**Highlights:**
- A Table valued function thate takes in 2 parameters
- A CTE is applied for readability and maintance
- Multiple Inner Joins are done to obtain the product category that a customer made
- A group by cluase is provided to be able to crunch numbers on the profit made under this category.
- The combination of Order by and TOP enables us to get the "Top customers" in this category which is powerful enough to do other queries if we want to contact these customers.

In [ ]:
-- TQ2 Q18

USE WideWorldImporters;
GO

CREATE OR ALTER FUNCTION [dbo].[topNPurchasesOnCategory] 
    (@desiredCategory NVARCHAR(100), @topNOrders INT)
RETURNS TABLE
AS
RETURN 
(
    WITH [TopOrdersUnderProductCategory] AS 
    (
        SELECT 
        TOP (@topNOrders) 
            C.CustomerId,
            O.OrderID,
            SC.SupplierCategoryName,
            SUM(OL.Quantity * OL.UnitPrice) AS TotalAmountSpent
        FROM 
            [Sales].[Customers] AS C
        INNER JOIN 
            [Sales].[Orders] AS O ON C.CustomerID = O.CustomerID
        INNER JOIN 
            [Sales].[OrderLines] AS OL ON OL.OrderID = O.OrderID
        INNER JOIN 
            [Purchasing].[PurchaseOrderLines] AS POL ON POL.PurchaseOrderID = O.OrderID
        INNER JOIN 
            [Warehouse].[StockItems] AS SI ON SI.StockItemID = POL.StockItemID
        INNER JOIN 
            [Purchasing].[Suppliers] AS S ON S.SupplierID = SI.SupplierID
        INNER JOIN 
            [Purchasing].[SupplierCategories] AS SC ON SC.SupplierCategoryID = S.SupplierCategoryID
        WHERE
            SC.SupplierCategoryName = @desiredCategory
        GROUP BY 
            C.CustomerID, O.OrderID, S.SupplierID, SC.SupplierCategoryName
        ORDER BY
            TotalAmountSpent DESC
    )
    SELECT
        CustomerID,
        OrderID,
        SupplierCategoryName,
        TotalAmountSpent
    FROM 
        TopOrdersUnderProductCategory
)
GO

***

**<u>TQ 3</u>**

**Proposition:**
jhh
We want to send a special discount to customers who have been in the top $$M_{I}$$  of each $$N_{I}$$ category. However we would also like to know a specific trhreshold for who to send a discount to. A number greater or equal to 2 would be a great criteria to find. For example we would like to send discounts who purchased from a (Novelty Goods Supplier) they were in the top 500, the same customer should be have purchased from a (Toy Supplier) and was in the top 600. This same customer has bought from a (Clothing Supplier) and was in the top 700. And then Find the intersection of this result set and then determine based on a count who should be given the discount.

It i possible to have customers who only made purchases from one supplier if the dataset is small.

**HighLights:**

- A virtual table idenitified as a CTE that cleverly uses Union all to capture the essence of a virtual table.
- Then once this table is generated it applies a custom function using the table as input! This effectively acts as a dynamic union operator without explicilty writing out.
- Once all of the of the tables have been added into the result set we will find out who should be emailed the discount
- We cant dynamically take the intersection so we will count the membership of the customer respective with the number of supplier fields they have participated in. This in effects checks the cardinality of the membership of a customer which we will check if it is greater or equal to the discount criteria.
- Once we find those customers we will find customers who are not companies (we want to target normal everyday customers). via the final where clause
- We also want to find meta data of the customer such as their name, their email and the total amount they spent across the categories.
- Multiple CTEs were created which speak for themselves.



In [ ]:
-- TQ 3 , Q20
USE WideWorldImporters;

DECLARE @LoyalCustomerThreshold AS INT = 2;

-- Define the criteria we are after
WITH [TopCustomerCriteria] AS (
    SELECT 'Novelty Goods Supplier' AS Category, 700 AS TopN
    UNION ALL
    SELECT 'Toy Supplier', 800
    UNION ALL
    SELECT 'Clothing Supplier', 900
    UNION ALL
    SELECT 'Packaging Supplier', 500
),
-- Add those customers using the custom function that enables us to find top customers
[AllCustomersWHoMatchedCriteria] AS
(
    SELECT  
        *
    FROM 
        TopCustomerCriteria  AS TCC
    CROSS APPLY 
        [dbo].[topNPurchasesOnCategory](TCC.Category, TCC.TopN)
),
-- Grouping and then counting shows the cardinality of membership which can be applied with the loyalCustomerThreshold
[LoyalCustomers] AS (
    SELECT 
        CustomerID
    FROM 
        AllCustomersWHoMatchedCriteria AS CWM
    GROUP BY 
        CustomerID
    HAVING 
        COUNT(DISTINCT Category) >= @LoyalCustomerThreshold
)

SELECT 
    C.CustomerName,
    P.EmailAddress,
    (
        SELECT SUM(CWM.TotalAmountSpent)
        FROM AllCustomersWHoMatchedCriteria AS CWM
        INNER JOIN LoyalCustomers AS C ON C.CustomerID = CWM.CustomerID
        GROUP BY C.CustomerID
        HAVING C.CustomerID = LC.CustomerID
    ) AS totalAmountSpent
FROM 
    LoyalCustomers AS LC
INNER JOIN 
    [Sales].[Customers] AS C ON C.CustomerID = LC.CustomerID
INNER JOIN 
    [Application].[People] AS P ON P.PersonID = C.PrimaryContactPersonID
WHERE 
    C.CustomerName NOT LIKE '%(%'
ORDER BY 
    totalAmountSpent DESC
;

***

# Presentation Queries TOP Worst with Corrections

**<u>TW 1</u>**

Q11

Proposition:

Find employees who have not been audited Give information about their Hire date and name

Highlights:

CTE that uses a inner set operator to find all employees who have yet to be audited with Except
Inner Join On Employee and CTE to find data about the employees
Coalesce function that compresses a subquery on the employees to find what department they belong too.


In [ ]:
-- TW 1 , Q11 

USE Northwinds2022TSQLV7;

WITH [NonAuditEmployees] AS
(
    (
        SELECT 
            DISTINCT EmployeeId
        FROM 
            [HumanResources].[Employee]
    )

    EXCEPT 

    (
        SELECT
            DISTINCT EmployeeId
        FROM 
            [Triggered].[AuditTriggeredEmployeeHistory] AS A
    )
)
SELECT 
    E.EmployeeFirstName,
    E.EmployeeLastName,
    E.HireDate,
    COALESCE((
        SELECT Department
        FROM [SystemVersioned].[Employee] AS SE 
        WHERE SE.EmployeeId = NAE.EmployeeId
    ), 'Unknown') AS DEPT
FROM 
    NonAuditEmployees AS NAE
INNER JOIN 
    [HumanResources].[Employee] AS E ON E.EmployeeId = NAE.EmployeeId


**Correction Notes**

This correct query now uses left joins instead of set operators. When doing a left join and checking for nulls we essentially discard where no matches (or intersection has occured). And also outside a left join is done to simplify the coalesce function 

In [ ]:
-- TW 1 Corrected 

USE Northwinds2022TSQLV7;

WITH NonAuditEmployees AS
(
    SELECT DISTINCT E.EmployeeId
    FROM [HumanResources].[Employee] AS E
    LEFT JOIN [Triggered].[AuditTriggeredEmployeeHistory] AS A ON E.EmployeeId = A.EmployeeId
    WHERE A.EmployeeId IS NULL
)
SELECT 
    E.EmployeeFirstName,
    E.EmployeeLastName,
    E.HireDate,
    COALESCE(SE.Department, 'Unknown') AS DEPT
FROM 
    NonAuditEmployees AS NAE
INNER JOIN 
    [HumanResources].[Employee] AS E ON E.EmployeeId = NAE.EmployeeId
LEFT JOIN 
    [SystemVersioned].[Employee] AS SE ON SE.EmployeeId = NAE.EmployeeId;


**<u>TW 2</u>**


**Proposition:**

Create a CTE that captures the relationship between Sales.Order and Sales.OrderDetail. This CTE should display all values of Order and the total quantity for each orderid as well as the total sum for each product id. Label this CTE as OrderValues. Using this same CTE find ther orderid customerId the value and its percentage against every other product that was bought by the same customer

**Highlights:**
- Common Table expression: To create a custom virtual table that summaraizes the relationship between Order and orderdetail
- Inner JOIN
- Subqueries using the same CTE that uses a aggregate function to summarize a value upon this self reference.
- The reusability of this CTE shows a huge benefit of using them in the first place.



In [ ]:
-- TW2, Q4 
USE Northwinds2022TSQLV7;

-- A common table expresion that summeraizes the relationship between Order and order detail.
--      That has values of any order and actual values associated with it.
WITH [OrderValues] AS
(
    SELECT 
        O.OrderId,
        O.CustomerId,
        O.EmployeeId,
        O.ShipperId,
        O.OrderDate,
        O.RequiredDate,
        O.ShipToDate,
        SUM(OD.Quantity) AS qty,
        CAST( SUM( OD.Quantity * OD.UnitPrice * (1 - OD.DiscountPercentage) ) AS numeric(12, 2) ) AS val
    FROM 
        [Sales].[Order] AS O
    INNER JOIN 
        [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId
    GROUP BY
        O.OrderId,
        O.CustomerId,
        O.EmployeeId,
        O.ShipperId,
        O.OrderDate,
        O.RequiredDate,
        O.ShipToDate
) 
SELECT 
    OrderId, 
    CustomerId, 
    val, 
    CAST(100.0 * val / (SELECT SUM(O2.val)
                        FROM OrderValues AS O2
                        WHERE O1.CustomerId = O2.CustomerId) 
        AS numeric(5,2)) AS valInPercent
FROM OrderValues as O1;

**Corrected Notes**
- Renaming for readability e.g. OrderValues to OrderSummary or qty to TotalQunatity
- Used a seperate table expression to calculate the valuePercentage instead of running a subquery on the the origincal CTE


In [ ]:
-- TW2 Corrected

USE Northwinds2022TSQLV7;

WITH OrderSummary AS (
    SELECT 
        O.OrderId,
        O.CustomerId,
        O.EmployeeId,
        O.ShipperId,
        O.OrderDate,
        O.RequiredDate,
        O.ShipToDate,
        SUM(OD.Quantity) AS TotalQuantity,
        CAST(SUM(OD.Quantity * OD.UnitPrice * (1 - OD.DiscountPercentage)) AS numeric(12, 2)) AS TotalValue
    FROM 
        [Sales].[Order] AS O
    INNER JOIN 
        [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId
    GROUP BY
        O.OrderId,
        O.CustomerId,
        O.EmployeeId,
        O.ShipperId,
        O.OrderDate,
        O.RequiredDate,
        O.ShipToDate
) 
SELECT 
    OS.OrderId, 
    OS.CustomerId, 
    OS.TotalValue AS OrderValue, 
    CAST(100.0 * OS.TotalValue / TotalCustomerValue.TotalValue AS numeric(5,2)) AS ValuePercentage
FROM 
    OrderSummary AS OS
INNER JOIN 
    (
        SELECT 
            CustomerId, 
            SUM(TotalValue) AS TotalValue
        FROM 
            OrderSummary
        GROUP BY 
            CustomerId
    ) AS TotalCustomerValue ON OS.CustomerId = TotalCustomerValue.CustomerId;


**<u>TW 3</u>**

**Proposition:**

Find the Maximum paid employees in each department

**Highlights:**
- A CTE that searches for the maximum salary in each department
- A CTE that uses the previous CTE and returns the name and Department and the top salary that the employee has
- The inner join finds the exact employee based on departmnent and salary

In [ ]:
-- TW3, Q12 

USE Northwinds2022TSQLV7;

WITH MaxSalariesPerDepartment AS (
    SELECT 
        E.Department,
        MAX(E.Salary) AS topSalary
    FROM 
        [SystemVersioned].[Employee] AS E
    GROUP BY
        E.Department
),
EmployeeDigest AS (
    SELECT 
        E.EmployeeFullName,
        MS.Department,
        MS.topSalary
    FROM 
        [SystemVersioned].[Employee] AS E
    INNER JOIN 
        MaxSalariesPerDepartment AS MS ON 
            MS.Department = E.Department AND
            E.Salary = MS.topSalary
)
SELECT *
FROM EmployeeDigest;


**Correction Note**

- General small formating
- Explicit selection of columns
- Better aliasing such as EmployeeName and MaxSalary.

In [ ]:
-- TW3 Corrected

USE Northwinds2022TSQLV7;

WITH MaxSalariesPerDepartment AS (
    SELECT 
        E.Department,
        MAX(E.Salary) AS MaxSalary
    FROM 
        [SystemVersioned].[Employee] AS E
    GROUP BY
        E.Department
),
EmployeeDigest AS (
    SELECT 
        E.EmployeeFullName AS EmployeeName,
        MS.Department,
        MS.MaxSalary AS TopSalary
    FROM 
        [SystemVersioned].[Employee] AS E
    INNER JOIN 
        MaxSalariesPerDepartment AS MS ON 
            MS.Department = E.Department AND
            E.Salary = MS.MaxSalary
)
SELECT 
    EmployeeName,
    Department,
    TopSalary
FROM 
    EmployeeDigest;
